<a href="https://colab.research.google.com/github/kashishjp/123vac/blob/main/Symptom-disease_prediction_severity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.6 MB/s eta 0:00:00


In [2]:
!pip install datasets


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 18.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from datasets import Dataset

def process_csv(file_path):
    df = pd.read_csv(file_path)
    qa_pairs = []

    for index, row in df.iterrows():
        question = row['Symptom']
        answer = row['weight']
        qa_pairs.append(f"Question: {question}\nAnswer: {answer}\n")

    return qa_pairs

def load_dataset(file_path, tokenizer):
    qa_pairs = process_csv(file_path)
    tokenized_dataset = tokenizer(qa_pairs, truncation=True, 
                                  padding='max_length', max_length=128,
                                  return_tensors="pt")
    dataset = Dataset.from_dict(tokenized_dataset)
    return dataset

# Load the pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained(model_name)

# Load and preprocess the dataset
train_dataset = load_dataset("Symptom-severity.csv", tokenizer)
valid_dataset = load_dataset("Symptom-severity.csv", tokenizer)

# Configure and train the model using the Trainer class
training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_steps=100,
    save_steps=100,
    warmup_steps=0,
    logging_dir="logs",
    evaluation_strategy="steps",
    save_total_limit=3,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)

trainer.train()


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=51, training_loss=2.0367840037626377, metrics={'train_runtime': 17.9953, 'train_samples_per_second': 22.172, 'train_steps_per_second': 2.834, 'total_flos': 26063880192000.0, 'train_loss': 2.0367840037626377, 'epoch': 3.0})

In [4]:
# Save the fine-tuned model
model.save_pretrained("fine_tuned_BhagavatGita_gpt2")


In [5]:

# Load the fine-tuned model
fine_tuned_model = GPT2LMHeadModel.from_pretrained("fine_tuned_BhagavatGita_gpt2")

def ask_question(question, model, tokenizer, max_length=128, num_return_sequences=1):
    prompt = f"Question: {question}\nAnswer:"
    inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True)
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=3,
        do_sample=True,
        temperature=1.0,
        top_k=50,
        top_p=0.9,
        early_stopping=True,
    )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = answer.replace(prompt, "").strip()
    
    # Truncate the answer after the first newline character
    answer = answer.split("\n")[0]
    
    return answer


# Ask questions using the fine-tuned model
question = "itching"
answer = ask_question(question, fine_tuned_model, tokenizer)
print(f"Question: {question}\nAnswer: {answer}")

question = "skin_rash"
answer = ask_question(question, fine_tuned_model, tokenizer)
print(f"Question: {question}\nAnswer: {answer}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: itching
Answer: 4
Question: skin_rash
Answer: 5


In [6]:

question = "chills"
answer = ask_question(question, fine_tuned_model, tokenizer)
print(f"Question: {question}\nAnswer: {answer}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: chills
Answer: 6
